In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pyts.image import RecurrencePlot, GramianAngularField
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Sequential
import json
import time
from collections import deque
import warnings
from pickle import dump, load

file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [2]:
currencies = ['Usd_Chf', 'Gbp_Usd', 'Usd_Cad', 'Aud_Usd', 'Eur_Usd', 'Usd_Jpy', 'Nzd_Usd']
years = '2016-2022'

dfs, df_longs = [], []

for currency_pair in currencies:
    df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
    df.Date = pd.to_datetime(df.Date)
    df.reset_index(drop=True, inplace=True)
    dfs.append(df)

    df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_{years}.csv')
    df_long.Date = pd.to_datetime(df_long.Date)
    df_long.reset_index(drop=True, inplace=True)
    df_longs.append(df_long)

In [3]:
for df in dfs:
    df['sin_hour'] = np.sin(2 * np.pi * df['Date'].dt.hour / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['Date'].dt.hour / 24)
    df['sin_day'] = np.sin(2 * np.pi * df['Date'].dt.day / 7)
    df['cos_day'] = np.cos(2 * np.pi * df['Date'].dt.day / 7)

for df_long in df_longs:
    df_long['sin_hour'] = np.sin(2 * np.pi * df_long['Date'].dt.hour / 24)
    df_long['cos_hour'] = np.cos(2 * np.pi * df_long['Date'].dt.hour / 24)
    df_long['sin_day'] = np.sin(2 * np.pi * df_long['Date'].dt.day / 7)
    df_long['cos_day'] = np.cos(2 * np.pi * df_long['Date'].dt.day / 7)

In [4]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

In [5]:
bid_ask_mid_prices_list = []

for df in dfs:
    # Add technical indicators (for additional features)
    df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
    df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
    df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()

    df['atr'] = atr(df)
    df['rsi'] = rsi(df)
    df['rsi_sma'] = df['rsi'].rolling(50).mean()
    df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
    df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
    df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
    df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

    df['sar'] = psar(df)

    df['chop36'] = chop(df, lookback=36)

    df['vo'] = vo(df['Volume'], short_lookback=18, long_lookback=36)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Extract the bid and ask prices and fractals and remove them from the df
    bid_ask_mid_prices = df[['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
    df.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close'], axis=1, inplace=True)
    bid_ask_mid_prices_list.append(bid_ask_mid_prices)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

for df_long in df_longs:
    df_long['ema200'] = pd.Series.ewm(df_long['Mid_Close'], span=200).mean()
    df_long['ema100'] = pd.Series.ewm(df_long['Mid_Close'], span=100).mean()
    df_long['ema50'] = pd.Series.ewm(df_long['Mid_Close'], span=50).mean()

    df_long['atr'] = atr(df_long)
    df_long['rsi'] = rsi(df_long)
    df_long['rsi_sma'] = df_long['rsi'].rolling(50).mean()
    df_long['adx'] = adx(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])

    df_long['chop36'] = chop(df_long, lookback=36)

    df_long['vo'] = vo(df_long['Volume'], short_lookback=18, long_lookback=36)

    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)

    df_long.drop(['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close'], axis=1, inplace=True)

    df_long.dropna(inplace=True)
    df_long.reset_index(drop=True, inplace=True)

In [6]:
look_back_size = 150

In [7]:
def grab_image_data(subset):
  # rp_transformer = RecurrencePlot()
  # rp_subset = rp_transformer.transform(subset)

  # return rp_subset

  gasf_transformer = GramianAngularField(method='summation')
  gasf_subset = gasf_transformer.transform(subset)

  return gasf_subset

  # gadf_transformer = GramianAngularField(method='difference')
  # gadf_subset = gadf_transformer.transform(subset)

  # return gadf_subset

  # image_data = np.append(rp_subset, gasf_subset, axis=-1)
  # image_data = np.append(image_data, gadf_subset ,axis=-1)
  
  # return image_data

In [15]:
buys_list = []
sells_list = []
nones_list = []

value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
spread_cutoff = 0.20
chop_cutoff = 0.50
vo_cutoff = 0.0
std_mult = 2.0
lookback = 36
risk_reward_ratio = 1.5

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

for idx in range(len(currencies)):
    currency_pair, df, df_long, bid_ask_mid_prices = currencies[idx], dfs[idx], df_longs[idx], bid_ask_mid_prices_list[idx]
    rounding = 3 if 'Jpy' in currency_pair else 5
    buys, sells, nones = [], [], []
    trade, prev_year = None, None

    print(f'RUNNING SIMULATION FOR {currency_pair}...')

    for i in range(look_back_size, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        if prev_year is None or curr_date.year > prev_year:
            prev_year = curr_date.year
            print(prev_year)
        curr_long = df_long.loc[df_long['Date'] < curr_date]
        if len(curr_long) < 2:
            continue
        ema200_2, ema100_2, ema50_2, atr2, sar2, rsi2, rsi_sma2, vo2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'ema50', 'atr', 'sar', 'rsi', 'rsi_sma', 'vo']]
        ema200_1, ema100_1, ema50_1, atr1, sar1, rsi1, rsi_sma1, vo1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'atr', 'sar', 'rsi', 'rsi_sma', 'vo']]
        mid_open2, mid_close2, mid_low2, mid_high2 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 2], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        mid_open1, mid_close1, mid_low1, mid_high1 = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i - 1], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High']]
        chop_long = curr_long.loc[curr_long.index[-2], 'chop36']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]

        emas_buy_signal = ema200_1 < ema100_1
        emas_sell_signal = ema200_1 > ema100_1

        macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
        macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

        sar_buy_signal = sar1 < min([mid_low1, mid_high1])
        sar_sell_signal = sar1 > max([mid_low1, mid_high1])

        rsi_buy_signal = rsi1 > rsi_sma1 and rsi1 > 50
        rsi_sell_signal = rsi1 < rsi_sma1 and rsi1 < 50

        chop_signal = chop_long > chop_cutoff

        bullish_candle = mid_close1 > mid_open1
        bearish_candle = mid_close1 < mid_open1

        mid_highs = list(df.loc[df.index[i - 38:i - 2], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - 38:i - 2], 'Mid_Low'])
        
        differences = []

        for z in range(len(mid_highs)):
            differences.append(abs(mid_highs[z] - mid_lows[z]))

        mean, std = np.array(differences).mean(), np.array(differences).std()

        bars_small_enough = abs(mid_high1 - mid_low1) <= mean + (std_mult * std) and abs(mid_high2 - mid_low2) <= mean + (std_mult * std)

        buy_limit = curr_ask_low <= curr_ao
        sell_limit = curr_bo <= curr_bid_high

        vo_signal = (vo2 + vo1) / 2 > vo_cutoff

        if trade is None:
            if macd_buy_signal and emas_buy_signal and sar_buy_signal and rsi_buy_signal and chop_signal and bullish_candle and bars_small_enough and buy_limit and vo_signal:
                open_price = float(curr_ao)
                pullback = sar1 - spread

                stop_loss = round(pullback, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

            elif macd_sell_signal and emas_sell_signal and sar_sell_signal and rsi_sell_signal and chop_signal and bearish_candle and bars_small_enough and sell_limit and vo_signal:
                open_price = float(curr_bo)
                pullback = sar1 + spread

                stop_loss = round(pullback, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)

                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

        if trade is not None:
            for j in range(i, len(df)):
                curr_date = df.loc[df.index[j], 'Date']
                curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = bid_ask_mid_prices.loc[bid_ask_mid_prices.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]

                if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                    nones.append(trade['start_date'])  

                    trade = None
                    break

                if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
                    trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                    day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                    if trade_amount + day_fees > 0:
                        buys.append(trade['start_date'])

                    trade = None
                    break

                if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                    nones.append(trade['start_date'])

                    trade = None
                    break

                if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
                    trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                    day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                    if trade_amount + day_fees > 0:
                        sells.append(trade['start_date'])

                    trade = None
                    break
    
    buys_list.append(buys)
    sells_list.append(sells)
    nones_list.append(nones)

    print(f'Buys: {len(buys)}')
    print(f'Sells: {len(sells)}')
    print(f'Nones: {len(nones)}\n')

RUNNING SIMULATION FOR Usd_Chf...
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Buys: 73
Sells: 62
Nones: 196

RUNNING SIMULATION FOR Gbp_Usd...
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Buys: 78
Sells: 71
Nones: 225

RUNNING SIMULATION FOR Usd_Cad...
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Buys: 63
Sells: 80
Nones: 283

RUNNING SIMULATION FOR Aud_Usd...
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Buys: 70
Sells: 59
Nones: 255

RUNNING SIMULATION FOR Eur_Usd...
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Buys: 79
Sells: 64
Nones: 218

RUNNING SIMULATION FOR Usd_Jpy...
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Buys: 75
Sells: 57
Nones: 242

RUNNING SIMULATION FOR Nzd_Usd...
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Buys: 65
Sells: 60
Nones: 300



In [11]:
buy_indices_list = []
sell_indices_list = []
nones_indices_list = []

for i in range(len(dfs) - 1):
    buys, sells, nones = buys_list[i], sells_list[i], nones_list[i]
    
    buy_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in buys]
    sell_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in sells]
    nones_indices = [df.index[df['Date'] == curr_date] - 1 for curr_date in nones]

    buy_indices_list.append(buy_indices)
    sell_indices_list.append(sell_indices)
    nones_indices_list.append(nones_indices)

    print(len(buy_indices))
    print(len(sell_indices))
    print(len(nones_indices))

test_df = dfs[-1]
test_buy_indices = [test_df.index[test_df['Date'] == curr_date] - 1 for curr_date in buys]
test_sell_indices = [test_df.index[test_df['Date'] == curr_date] - 1 for curr_date in sells]
test_nones_indices = [test_df.index[test_df['Date'] == curr_date] - 1 for curr_date in nones]

print(len(test_buy_indices))
print(len(test_sell_indices))
print(len(test_nones_indices))

264
254
956
194
222
740
194
222
740


In [12]:
for df in dfs:
    df.drop('Date', axis=1, inplace=True)

In [25]:
def get_sequential_data():
    no_actions = []
    buys = []
    sells = []

    test_no_actions = []
    test_buys = []
    test_sells = []

    for z in range(len(dfs)):
        df = dfs[z]

        if z < len(dfs) - 1:
            buy_indices, sell_indices, nones_indices = buy_indices_list[z], sell_indices_list[z], nones_indices_list[z]

        else:
            buy_indices, sell_indices, nones_indices = test_buy_indices, test_sell_indices, test_nones_indices

        for i in buy_indices:
            if len(i) == 1:
                i = i[0]
                seq = df.iloc[i - look_back_size + 1:i + 1, :]

                if seq.shape == (look_back_size, 20):
                    seq = grab_image_data(seq)

                    if z < len(dfs) - 1:
                        buys.append([seq, np.array([0, 1, 0])])

                    else:
                        test_buys.append([seq, np.array([0, 1, 0])])

        for i in sell_indices:
            if len(i) == 1:
                i = i[0]
                seq = df.iloc[i - look_back_size + 1:i + 1, :]

                if seq.shape == (look_back_size, 20):
                    seq = grab_image_data(seq)

                    if z < len(dfs) - 1:
                        sells.append([seq, np.array([0, 0, 1])])

                    else:
                        test_sells.append([seq, np.array([0, 0, 1])])

        for i in nones_indices:
            if len(i) == 1:
                i = i[0]
                seq = df.iloc[i - look_back_size + 1:i + 1, :]

                if seq.shape == (look_back_size, 20):
                    seq = grab_image_data(seq)

                    if z < len(dfs) - 1:
                        no_actions.append([seq, np.array([1, 0, 0])])

                    else:
                        test_no_actions.append([seq, np.array([1, 0, 0])])

    np.random.shuffle(no_actions)
    np.random.shuffle(buys)
    np.random.shuffle(sells)

    lower = min(len(no_actions), len(buys), len(sells))

    no_actions = no_actions[:int(lower * 2)]

    sequential_data = no_actions + buys + sells
    np.random.shuffle(sequential_data)

    np.random.shuffle(test_no_actions)
    np.random.shuffle(test_buys)
    np.random.shuffle(test_sells)

    lower = min(len(test_no_actions), len(test_buys), len(test_sells))

    test_no_actions = test_no_actions[:int(lower * 2)]

    test_sequential_data = test_no_actions + test_buys + test_sells
    np.random.shuffle(test_sequential_data)

    return sequential_data, test_sequential_data

In [26]:
sequential_data, test_sequential_data = get_sequential_data()

In [27]:
len(sequential_data)

1847

In [28]:
len(test_sequential_data)

804

In [29]:
training_proportion = 0.70
train_test_cutoff_index = int(len(sequential_data) * training_proportion)

train_set = sequential_data[0:train_test_cutoff_index]
test_set = sequential_data[train_test_cutoff_index:]

print('Dataset shapes:')
print(len(train_set))
print(len(test_set))

Dataset shapes:
1292
555


In [30]:
x_train = []
y_train = []

for seq, target in train_set:
  x_train.append(seq)
  y_train.append(target)

x_test = []
y_test = []

for seq, target in test_set:
  x_test.append(seq)
  y_test.append(target)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [31]:
x_train.shape

(1292, 200, 20, 20)

In [32]:
y_train.shape

(1292, 3)

In [33]:
# Number of possible actions to take - determines the output dimension of the
#  neural network
n_actions = 3
input_data_shape = x_train.shape[1:]

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3), padding ='Same', 
                 activation ='relu', input_shape = input_data_shape))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(n_actions, activation = "softmax"))

In [34]:
# Hyperparameters
n_epochs = 200
batch_size = 16
n_steps = len(x_train) // batch_size 
mean_loss = tf.keras.metrics.Mean() 
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.categorical_crossentropy
metrics = [tf.keras.metrics.CategoricalAccuracy()]
# loss_fn = tf.keras.losses.MeanAbsoluteError
# metrics = [tf.keras.metrics.MeanAbsoluteError()]


average_training_losses = []
average_test_accuracies = []

In [35]:
early_stop = EarlyStopping(monitor='val_accuracy', verbose=1, patience=n_epochs)
model_checkpoint = ModelCheckpoint('/Users/mymac/forex_macd_cnn', monitor='val_accuracy', save_best_only=True, verbose=1)

optimizer = Adam()

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [36]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stop, model_checkpoint]
)

Epoch 1/200
81/81 [==============================] - ETA: 0s - loss: 2.0331 - accuracy: 0.4698
Epoch 00001: val_accuracy improved from -inf to 0.49369, saving model to /Users/mymac/forex_macd_cnn
INFO:tensorflow:Assets written to: /Users/mymac/forex_macd_cnn/assets
81/81 [==============================] - 7s 88ms/step - loss: 2.0331 - accuracy: 0.4698 - val_loss: 1.0580 - val_accuracy: 0.4937
Epoch 2/200
81/81 [==============================] - ETA: 0s - loss: 1.0540 - accuracy: 0.4954
Epoch 00002: val_accuracy did not improve from 0.49369
81/81 [==============================] - 6s 78ms/step - loss: 1.0540 - accuracy: 0.4954 - val_loss: 1.0416 - val_accuracy: 0.4937
Epoch 3/200
81/81 [==============================] - ETA: 0s - loss: 1.0504 - accuracy: 0.4938
Epoch 00003: val_accuracy did not improve from 0.49369
81/81 [==============================] - 6s 80ms/step - loss: 1.0504 - accuracy: 0.4938 - val_loss: 1.0616 - val_accuracy: 0.4937
Epoch 4/200
81/81 [=========================

KeyboardInterrupt: 

In [62]:
model = load_model('/Users/mymac/forex_macd_cnn')

test_df = dfs[-1]
test_bid_ask_mid_prices = bid_ask_mid_prices_list[-1]
x_val_test = []
y_val_test = []

for seq, target in test_sequential_data:
    x_val_test.append(seq)
    y_val_test.append(target)

In [63]:
loss, accuracy = model.evaluate(np.array(x_val_test), np.array(y_val_test))

print(loss)
print(accuracy)

42/42 [==============================] - 2s 53ms/step - loss: 1.0452 - accuracy: 0.4815
1.0451610088348389
0.48145344853401184
